This file is just to test the connection between locally installed mistrial with ollama and python


In [1]:
import ollama

# Run a simple prompt
response = ollama.chat(model='mistral', messages=[
  {'role': 'user', 'content': 'What is the capital of France?'}
])

print(response['message']['content'])


 The capital city of France is Paris.


A bit detailed example on how to use LLM for batch inputs

In [2]:
import ollama
import time
import json

# --- Step 1: Define your batch inputs with slight variations ---
base_prompt = "Summarize the importance of {} in human life."
keywords = ["patience", "gratitude", "forgiveness", "hope", "resilience"]

# Create structured input list
batch_inputs = [base_prompt.format(word) for word in keywords]

# --- Step 2: Store results ---
results = []

# --- Step 3: Loop through batch and query Ollama ---
for idx, prompt in enumerate(batch_inputs):
    print(f"\n[Prompt {idx+1}] -> {prompt}")
    start_time = time.time()

    response = ollama.chat(
        model='mistral',
        messages=[
            {'role': 'user', 'content': prompt}
        ]
    )

    end_time = time.time()
    elapsed = round(end_time - start_time, 2)

    structured = {
        "prompt": prompt,
        "response": response['message']['content'],
        "response_time_seconds": elapsed
    }

    results.append(structured)

# --- Step 4: Print final results in JSON format ---
print("\n=== Final Results ===")
print(json.dumps(results, indent=2))



[Prompt 1] -> Summarize the importance of patience in human life.

[Prompt 2] -> Summarize the importance of gratitude in human life.

[Prompt 3] -> Summarize the importance of forgiveness in human life.

[Prompt 4] -> Summarize the importance of hope in human life.

[Prompt 5] -> Summarize the importance of resilience in human life.

=== Final Results ===
[
  {
    "prompt": "Summarize the importance of patience in human life.",
    "response": " Patience plays a crucial role in human life, providing numerous benefits for personal growth and social interactions. Here's a brief summary:\n\n1. Personal Development: Cultivating patience allows individuals to navigate challenges more effectively by fostering resilience and perseverance. This skill is essential in overcoming obstacles and achieving long-term goals.\n\n2. Improved Emotional Regulation: Patience helps reduce stress, frustration, and anger, as it enables people to respond thoughtfully instead of reacting impulsively to diffi

Parallel execution of prompts

In [3]:
import ollama
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Step 1: Prepare inputs ---
base_prompt = "Summarize the importance of {} in human life."
keywords = ["patience", "gratitude", "forgiveness", "hope", "resilience", "trust", "kindness", "humility", "courage", "empathy"]
batch_inputs = [base_prompt.format(word) for word in keywords]

# --- Step 2: Define function for threaded execution ---
def query_ollama(prompt):
    start_time = time.time()
    try:
        response = ollama.chat(
            model='mistral',
            messages=[{'role': 'user', 'content': prompt}]
        )
        elapsed = round(time.time() - start_time, 2)
        return {
            "prompt": prompt,
            "response": response['message']['content'],
            "response_time_seconds": elapsed
        }
    except Exception as e:
        return {
            "prompt": prompt,
            "response": f"Error: {str(e)}",
            "response_time_seconds": None
        }

# --- Step 3: Execute in parallel ---
results = []
max_threads = 4  # Set based on your CPU capability

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    future_to_prompt = {executor.submit(query_ollama, prompt): prompt for prompt in batch_inputs}

    for future in as_completed(future_to_prompt):
        result = future.result()
        print(f"[Done] Prompt: {result['prompt']} | Time: {result['response_time_seconds']}s")
        results.append(result)

# --- Step 4: Output structured results ---
print("\n=== Final Results (JSON) ===")
print(json.dumps(results, indent=2))

# --- Optional: Save to file ---
with open("parallel_mistral_results.json", "w") as f:
    json.dump(results, f, indent=2)


[Done] Prompt: Summarize the importance of patience in human life. | Time: 24.51s
[Done] Prompt: Summarize the importance of gratitude in human life. | Time: 25.19s
[Done] Prompt: Summarize the importance of hope in human life. | Time: 34.62s
[Done] Prompt: Summarize the importance of forgiveness in human life. | Time: 41.73s
[Done] Prompt: Summarize the importance of trust in human life. | Time: 27.38s
[Done] Prompt: Summarize the importance of resilience in human life. | Time: 37.84s
[Done] Prompt: Summarize the importance of kindness in human life. | Time: 33.79s
[Done] Prompt: Summarize the importance of humility in human life. | Time: 33.88s
[Done] Prompt: Summarize the importance of courage in human life. | Time: 33.26s
[Done] Prompt: Summarize the importance of empathy in human life. | Time: 30.67s

=== Final Results (JSON) ===
[
  {
    "prompt": "Summarize the importance of patience in human life.",
    "response": " Patience is a crucial virtue in human life, offering several